# Kümeleme işlemi uygulanmış veri üzerinde eğitim ve ilgili metriklerin sunumu

In [16]:
%ls

 Volume in drive C has no label.
 Volume Serial Number is 7096-3171

 Directory of c:\Users\bugra\Desktop\fut_scout_data\fut_scout_data_operations\understat_classification

14.05.2022  19:41    <DIR>          .
14.05.2022  19:41    <DIR>          ..
14.05.2022  18:48               673 clustering.py
14.05.2022  19:38            27.695 clustering_plot.ipynb
14.05.2022  19:02             1.032 clustering_plot.py
14.05.2022  18:33    <DIR>          data
14.05.2022  18:33             1.615 data_append.py
14.05.2022  19:41    <DIR>          figures
14.05.2022  18:33             2.477 full_data.py
14.05.2022  18:33    <DIR>          model
14.05.2022  19:22             4.295 train.py
14.05.2022  19:41            26.287 training_with_custom_parameters.ipynb
14.05.2022  19:41                 0 training_with_hyper_parameter_tuning.ipynb
14.05.2022  18:33    <DIR>          understat_classification
               8 File(s)         64.074 bytes
               6 Dir(s)  287.937.683.456 bytes free


## Kütüphanelerin eklenmesi

In [17]:

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn import metrics

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from keras import callbacks
import tensorflow as tf
import keras

## Kullanacağımız Yapay Sinir Ağı Modelinin Tanımlanması

In [18]:
def createModel(n_cols):

    model = keras.Sequential([

        keras.layers.Dense(units=128, activation='relu'),
        keras.layers.Dense(units=128, activation='relu'),
        keras.layers.Dense(units=64, activation='relu'),
        keras.layers.Dropout(0.25),
        keras.layers.Dense(units=32, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(units=8, activation='relu'),
        keras.layers.Dense(units=n_cols, activation='softmax'),
    ]
    )

    optim = Adam(lr=0.001)
    model.compile(optimizer=optim, loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

## KNN Eğitim Fonksiyonunun oluşturulması

In [19]:
def knn(X_train, y_train, X_test, y_test):
        plt.clf()
        dt = KNeighborsClassifier()
        dt.fit(X_train, y_train)

        y_pred = dt.predict(X_test)
        cnf_matrix = metrics.confusion_matrix(
                y_test.argmax(axis=1), y_pred.argmax(axis=1))
        p = sns.heatmap(pd.DataFrame(cnf_matrix),
                        annot=True, cmap="YlGnBu", fmt='g')
        plt.title('Confusion matrix', y=1.1)
        plt.ylabel('Actual label')
        plt.xlabel('Predicted label')

        plt.savefig('knn_default_parameters_cm.png')
        plt.clf()

## Decision Tree Eğitim Fonksiyonunun oluşturulması

In [20]:
def decision_tree(X_train, y_train, X_test, y_test):
        plt.clf()
        dt = DecisionTreeClassifier()
        dt.fit(X_train, y_train)

        y_pred = dt.predict(X_test)
        cnf_matrix = metrics.confusion_matrix(
                y_test.argmax(axis=1), y_pred.argmax(axis=1))
        p = sns.heatmap(pd.DataFrame(cnf_matrix),
                        annot=True, cmap="YlGnBu", fmt='g')
        plt.title('Confusion matrix', y=1.1)
        plt.ylabel('Actual label')
        plt.xlabel('Predicted label')

        plt.savefig('decision_tree_default_parameters_cm.png')
        plt.clf()

## ANN Eğitim Fonksiyonunun oluşturulması

In [21]:
def ann_model(X_train, y_train, X_test, y_test):
        early_stopping = callbacks.EarlyStopping(
        min_delta=0.00001,
        patience=12,
        restore_best_weights=True)
        model = createModel(n_cols=n_cols)

        history = model.fit(X_train, y_train, batch_size=128, epochs=1000, callbacks=[
                        early_stopping], validation_split=0.2)
        model.save('model/model_sc_pca.h5')
        plt.subplot(211)
        plt.title('Loss')
        plt.plot(history.history['loss'], label='train')
        plt.plot(history.history['val_loss'], label='test')
        plt.legend()
        
        plt.subplot(212)
        plt.title('Accuracy')
        plt.plot(history.history['accuracy'], label='train')
        plt.plot(history.history['val_accuracy'], label='test')
        plt.legend()
        plt.savefig('ann_default_parameters_acc_loss.png')
        plt.clf()
        y_pred = model.predict(X_test)
        cnf_matrix = metrics.confusion_matrix(
                y_test.argmax(axis=1), y_pred.argmax(axis=1))
        sns.heatmap(pd.DataFrame(cnf_matrix),
                        annot=True, cmap="YlGnBu", fmt='g')
        plt.title('Confusion matrix', y=1.1)
        plt.ylabel('Actual label')
        plt.xlabel('Predicted label')

        plt.savefig('ann_default_parameters_cm.png')
        plt.clf()

In [22]:
df = pd.read_csv('data/clustered_data.csv').drop(['id', 'player_name', 'position'], axis=1)
X = df.drop(['classes'], axis = 1).values
y = df['classes']
n_cols = len(np.unique(y))

sc = StandardScaler()
X = sc.fit_transform(X)

pca = PCA(2)
X = pca.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.36, random_state=14)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [23]:
knn(X_train, y_train, X_test, y_test)
decision_tree(X_train, y_train, X_test, y_test)
ann_model(X_train, y_train, X_test, y_test)

Epoch 1/1000


c:\Users\bugra\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


30/30 [==============================] - 1s 17ms/step - loss: 1.9017 - accuracy: 0.2743 - val_loss: 1.4163 - val_accuracy: 0.4106
Epoch 2/1000
30/30 [==============================] - 0s 11ms/step - loss: 1.4292 - accuracy: 0.4215 - val_loss: 1.1266 - val_accuracy: 0.5820
Epoch 3/1000
30/30 [==============================] - 0s 12ms/step - loss: 1.1811 - accuracy: 0.5293 - val_loss: 0.8825 - val_accuracy: 0.6656
Epoch 4/1000
30/30 [==============================] - 0s 11ms/step - loss: 0.9835 - accuracy: 0.6209 - val_loss: 0.7418 - val_accuracy: 0.7217
Epoch 5/1000
30/30 [==============================] - 0s 11ms/step - loss: 0.8751 - accuracy: 0.6675 - val_loss: 0.6189 - val_accuracy: 0.7799
Epoch 6/1000
30/30 [==============================] - 0s 11ms/step - loss: 0.7697 - accuracy: 0.7133 - val_loss: 0.5162 - val_accuracy: 0.8148
Epoch 7/1000
30/30 [==============================] - 0s 9ms/step - loss: 0.6760 - accuracy: 0.7424 - val_loss: 0.4314 - val_accuracy: 0.8212
Epoch 8/1000


<Figure size 432x288 with 0 Axes>